- A typical TensorFlow training input pipeline can be framed as an ETL process:

	- Extract: Read data from persistent storage -- either local (e.g. HDD or SSD) or remote (e.g. GCS or HDFS).
	- Transform: Use CPU cores to parse and perform preprocessing operations on the data such as image decompression, data augmentation transformations (such as random crop, flips, and color distortions), shuffling, and batching.
	- Load: Load the transformed data onto the accelerator device(s) (for example, GPU(s) or TPU(s)) that execute the machine learning model.

In [5]:
# When using the tf.estimator.Estimator API, the first two phases (Extract 
# and Transform) are captured in the input_fn passed to tf.estimator.Estimator.train.
def parse_fn(example):
  """Parse TFExample records and perform simple data augmentation."""
  example_fmt = {
    "image": tf.FixedLengthFeature((), tf.string, ""),
    "label": tf.FixedLengthFeature((), tf.int64, -1)
  }
  parsed = tf.parse_single_example(example, example_fmt)
  image = tf.image.decode_image(parsed["image"])
  image = _augment_helper(image)  # augments image using slice, reshape, resize_bilinear
  return image, parsed["label"]

def input_fn():
  files = tf.data.Dataset.list_files("/path/to/dataset/train-*.tfrecord")
  # 'interleave()' is a method that can be used to interleave (entrelazar) together 
  # elements from multiple datasets.
  # Example: preprocess 4 files concurrently, and interleave blocks of 16 records 
  # from each file.
  # dataset = files.interleave(tf.data.TFRecordDataset)
  dataset = files.apply(tf.contrib.data.parallel_interleave(
    tf.data.TFRecordDataset, cycle_length=FLAGS.num_parallel_readers))
  dataset = dataset.shuffle(buffer_size=FLAGS.shuffle_buffer_size)
  dataset = dataset.apply(tf.contrib.data.map_and_batch(
    map_func=parse_fn, batch_size=FLAGS.batch_size))
  dataset = dataset.prefetch(buffer_size=FLAGS.prefetch_buffer_size)
  return dataset

- To perform a training step, you must first extract and transform the training data and then feed it to a model running on an accelerator. However, in a naive synchronous implementation, while the CPU is preparing the data, the accelerator is sitting idle. Conversely, while the accelerator is training the model, the CPU is sitting idle. The training step time is thus the sum of both CPU pre-processing time and the accelerator training time.

- Pipelining overlaps the preprocessing and model execution of a training step. While the accelerator is performing training step N, the CPU is preparing the data for step N+1. Doing so reduces the step time to the maximum (as opposed to the sum) of the training and the time it takes to extract and transform the data.

- tf.data.Dataset.prefetch transformation, which can be used to decouple the time data is produced from the time it is consumed.

- Prefetch elements from the input dataset ahead of the time they are requested.

- prefetch(n) indicaes that a single training step consumes n batches.

- tf.data.Dataset.map transformation, which applies a user-defined function (for example, parse_fn from the running example) to each element of the input dataset.

- Because input elements are independent of one another, the pre-processing can be parallelized across multiple CPU cores, using the num_parallel_calls argument to specify the level of parallelism.

- Parallelizing the batch creation: the tf.data API provides the tf.contrib.data.map_and_batch transformation, which effectively "fuses" the map and batch transformations.

- tf.contrib.data.parallel_interleave transformation is used to parallelize the data extraction and interleave the contents of other datasets (such as data file readers). The number of datasets to overlap can be specified by the cycle_length argument.

- By default, the parallel_interleave transformation provides a deterministic ordering of elements to aid reproducibility.

- If map function does little work, we recommend vectorizing the user-defined function (that is, have it operate over a batch of inputs at once) and apply the batch transformation before the map transformation.

- If the user-defined function passed into the map transformation is expensive, apply the cache transformation after the map transformation as long as the resulting dataset can still fit into memory or local storage.

- If this user-defined function increases the space required to store the dataset beyond the cache capacity, consider pre-processing your data before your training job to reduce resource usage.

- The tf.data.Dataset.repeat transformation repeats the input data a finite (or infinite) number of times; each repetition of the data is typically referred to as an epoch.

- When possible, we recommend using the fused tf.contrib.data.shuffle_and_repeat transformation, which combines the best of both worlds (good performance and strong ordering guarantees). Otherwise, we recommend shuffling before repeating.

- Use the prefetch(n) transformation to overlap the work of a producer and consumer.

- Parallelize the map transformation by setting the num_parallel_calls(num_of_cpu_cores) argument.

- We recommend using the fused map_and_batch transformation.

- If you are working with data stored remotely and / or requiring deserialization, we recommend using the parallel_interleave transformation to overlap the reading (and deserialization) of data from different files.

- We recommend applying the shuffle transformation before the repeat transformation, ideally using the fused shuffle_and_repeat transformation.